<a href="https://colab.research.google.com/github/jl749/CJ_VRP/blob/master/YS/fix_folium_%ED%99%94%EC%84%B1%EC%8B%9C_marker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://blog.naver.com/PostView.nhn?blogId=kcchang61&logNo=221350672356

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geojson

In [ ]:
import folium
import geojson

with open('/content/drive/MyDrive/CJ/SIG_202101/TL_SCCO_SIG_WGS84.json') as f:
  gj = geojson.load(f)

features = gj['features'][52]  # 화성시

In [ ]:
print(len(features['geometry']['coordinates']))
print(len(features['geometry']['coordinates'][0]))
print(len(features['geometry']['coordinates'][0][0]))
print(features['geometry']['coordinates'][0])
print(features['geometry']['coordinates'][1])

2
454
2
[[127.037, 37.237], [127.023, 37.225], [126.989, 37.232], [126.97, 37.23], [126.958, 37.24], [126.947, 37.257], [126.931, 37.261], [126.928, 37.277], [126.915, 37.29], [126.901, 37.291], [126.89, 37.282], [126.862, 37.281], [126.864, 37.273], [126.845, 37.264], [126.837, 37.271], [126.829, 37.271], [126.81, 37.281], [126.81, 37.267], [126.796, 37.271], [126.791, 37.267], [126.79, 37.244], [126.771, 37.249], [126.758, 37.249], [126.745, 37.255], [126.734, 37.249], [126.723, 37.256], [126.716, 37.255], [126.687, 37.262], [126.677, 37.246], [126.665, 37.242], [126.667, 37.232], [126.677, 37.224], [126.666, 37.213], [126.673, 37.2], [126.665, 37.192], [126.665, 37.182], [126.655, 37.175], [126.661, 37.162], [126.67, 37.156], [126.683, 37.161], [126.695, 37.152], [126.683, 37.149], [126.679, 37.134], [126.685, 37.122], [126.684, 37.112], [126.757, 37.055], [126.746, 37.042], [126.751, 37.03], [126.789, 37.03], [126.795, 37.026], [126.798, 37.013], [126.808, 37.013], [126.811, 37.013

In [ ]:
features['geometry']['coordinates'][0][0][0] ,features['geometry']['coordinates'][0][0][1] 

(127.037, 37.237)

In [ ]:
# init map
m = folium.Map(location=[37.335, 126.714], tiles="OpenStreetMap", zoom_start=10)

m.choropleth(
geo_data=features,
name='choropleth',

columns=['Code', 'Population'],
key_on='feature.properties.SIG_CD',
fill_color='green',
fill_opacity=0.7,
line_opacity=0.5,
legend_name='Population Rate (%)'
)

folium.LayerControl().add_to(m)
display(m)

/usr/local/lib/python3.7/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


#위경도 데이터 저장후에 하나의 polygon 데이터 만들기 


In [ ]:
lons_vect = [] #경도
lats_vect = [] #위도


#위도 경도 데이터 저장 
for i in range(len(features['geometry']['coordinates'])):
  for j in range(len(features['geometry']['coordinates'][i])):
    lats_vect.append(features['geometry']['coordinates'][i][j][0])
    lons_vect.append(features['geometry']['coordinates'][i][j][1])


In [ ]:
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

lons_lats_vect = np.column_stack((lons_vect, lats_vect)) # Reshape coordinates
polygon = Polygon(lons_lats_vect) # create polygon
#lons_lats_vect
#polygon 좌표 안에 있는지 확인 하는 코드 

#polygon.contains(point) polygon 안에 point 가 포함되는지 확인  
#point.within(polygon)  point가 polygon 안에 있는지 확인
#혹시 몰라서 일단 두개다 만족하는지 확인하는 코드 
def contain_x_y(x,y):
  point = Point(x,y) # create point
  if polygon.contains(point) and point.within(polygon):
    return True
  return False 


True
True


#화성시 탐색 및 일정 거리 좌표 찍기
###화성시 맨위에 있는 위도 좌표 37.290941 북
###화성시 맨왼쪽에 있는 경도 좌표 126.657198 서
###화성시 맨 아래쪽 있는 위도 좌표 37.016417  남
###화성시 맨 오른쪽 있는 경도 좌표 127.161169 동


##Search
###시작점 (맨위위도 , 맨 왼쪽경도)
###끝점 (맨아래 , 맨 오른쪽 좌표 까지 ) 
### DISTANCE = 1KM 



In [ ]:
!pip install pyproj

     |████████████████████████████████| 6.3 MB 4.2 MB/s 


##좌표 Transfomer 를 지원하지 않는다... -> inf inf 계속 리턴...

In [59]:
import itertools

import folium
from pyproj import Transformer, transform

transformer_5186_4326 = Transformer.from_crs("epsg:5186","epsg:4326")
print(transformer_5186_4326)

p_4326 = (126.657198, 37.290941) # 화성시 북서쪽 왼쪽 가장자리 좌표
meters = 1000 # 1km
n = 50 # 50x50

#이점을 기준으로 좌표변경 값을 얻어내기 위해 transform 이 지원을 안함 
p_x_5186, p_y_5186 = transform(4326, 5186, p_4326[0], p_4326[1])

print(p_x_5186,p_y_5186)
geo_jsons = []
'''
for x, y in itertools.product(range(n), range(n)):
    grid_lower_l = transformer_5186_4326.transform(p_x_5186 + (x + 0) * meters, p_y_5186 + (y + 0) * meters)
    grid_lower_r = transformer_5186_4326.transform(p_x_5186 + (x + 1) * meters, p_y_5186 + (y + 0) * meters)
    grid_upper_r = transformer_5186_4326.transform(p_x_5186 + (x + 1) * meters, p_y_5186 + (y + 1) * meters)
    grid_upper_l = transformer_5186_4326.transform(p_x_5186 + (x + 0) * meters, p_y_5186 + (y + 1) * meters)

    print(grid_lower_l,grid_lower_r,grid_upper_l,grid_upper_r)
'''

proj=pipeline step proj=axisswap order=2,1 step inv proj=tmerc lat_0=38 lon_0=127 k=1 x_0=200000 y_0=600000 ellps=GRS80 step proj=unitconvert xy_in=rad xy_out=deg step proj=axisswap order=2,1
(126.657198, 37.290941)
inf inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  del sys.path[0]


'\nfor x, y in itertools.product(range(n), range(n)):\n    grid_lower_l = transformer_5186_4326.transform(p_x_5186 + (x + 0) * meters, p_y_5186 + (y + 0) * meters)\n    grid_lower_r = transformer_5186_4326.transform(p_x_5186 + (x + 1) * meters, p_y_5186 + (y + 0) * meters)\n    grid_upper_r = transformer_5186_4326.transform(p_x_5186 + (x + 1) * meters, p_y_5186 + (y + 1) * meters)\n    grid_upper_l = transformer_5186_4326.transform(p_x_5186 + (x + 0) * meters, p_y_5186 + (y + 1) * meters)\n\n    print(grid_lower_l,grid_lower_r,grid_upper_l,grid_upper_r)\n'